In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [92]:
data=pd.read_csv(r"C:\Users\rudra\Downloads\train.csv")
data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## Basic EDA

In [95]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [97]:
data.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [99]:
data.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


## Train/Test Split

In [106]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data.iloc[:,1:],data['Survived'],test_size=0.2,random_state=45)

In [108]:
x_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
332,1,male,38.0,0,1,153.4625,S
281,3,male,28.0,0,0,7.8542,S


## Handling Missing values and Encoding steps

In [111]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing  import MinMaxScaler
from sklearn.compose import ColumnTransformer

## Column Transformers

In [114]:
trf1=ColumnTransformer([
    ('impute_age',SimpleImputer(strategy='mean'),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [116]:
trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,dtype=np.int32,handle_unknown="ignore"),[1,6])
],remainder="passthrough")

In [118]:
trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))])

## Logistic Regression

In [121]:
from sklearn.linear_model import LogisticRegression
trf4=LogisticRegression()

## Import Pipeline

In [124]:
from sklearn.pipeline import Pipeline,make_pipeline
pipe=Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4)
])

In [126]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(dtype=<class 'numpy.int32'>,
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4', LogisticRegression())])

## Predict data

In [129]:
y_prd=pipe.predict(x_test)
df=pd.DataFrame({'Actual':y_test,"Predict":y_prd})
df.head()

,Actual,Predict
248,1,0
197,0,0
133,1,0
169,0,0
736,0,0


## Check Accuracy Score

In [132]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_prd)*100

73.18435754189943

In [83]:
score=[]
for i in range(1,100):
    x_train,x_test,y_train,y_test=train_test_split(data.iloc[:,1:],data['Survived'],test_size=0.2,random_state=i)
    pipe.fit(x_train,y_train)
    y_prd=pipe.predict(x_test)
    score.append(accuracy_score(y_test,y_prd)*100)

## Exporting The Pipeline

In [141]:
import pickle
pickle.dump(pipe,open('titanic_prediction.pkl','wb'))